In [1]:
from scripts.round3.figures_import_helper_r3 import *

In [2]:
list_data_sets = [
    #('Bridges', 'bc_collusion.csv'),
    #('Electric', 'ec_collusion.csv'),
    #('Pre-Stressed Concrete', 'pc_collusion.csv'),
    ('Floods', 'fc_collusion.csv')
]

In [5]:
file = 'fc_collusion.csv'
data = asymptotics.MultistagePIDMeanAuctionData(
os.path.join(path_data, file))
data_before = asymptotics.MultistagePIDMeanAuctionData.from_clean_bids(
data.df_bids.loc[data.data.before == 1])
data_after = asymptotics.MultistagePIDMeanAuctionData.from_clean_bids(
data.df_bids.loc[data.data.before.isnull()])

filter_ties = auction_data.FilterTies()
data_after = filter_ties(data_after)

print(file, '\n\n sample size\t', data_after.df_auctions.shape[0])
print(data_after.demand_vector(all_deviations), '\n')


metric = rebidding.EfficientMultistageIsNonCompetitive
metric.min_markup = .1
solver = asymptotics.AsymptoticMultistageSolver(
    data=data_after, 
    deviations=[-.02, 0, .001], 
    metric=metric, 
    plausibility_constraints=[environments.EmptyConstraint()],
    num_points=1000, moment_matrix=multistage_moment_matrix,                 
    confidence_level=.95, enhanced_guesses=True
)
print(' tolerances\n', solver.tolerance, '\n')

fc_collusion.csv 

 sample size	 82
win_down     0.230996
marg_cont    0.001220
marg_info    0.020562
win0         0.194822
win_up       0.194822
dtype: float64 

 tolerances
 [[-1.22719433e-01]
 [ 2.79291140e-02]
 [ 2.96571488e-01]
 [ 2.83700960e-04]
 [-9.30723368e-02]] 



In [6]:
pb = solver.problem
pb.problem.solve()

0.036569674219368

In [23]:
data_after.df_auctions['lowest'].describe()

count    68.000000
mean      0.922224
std       0.078077
min       0.641430
25%       0.900099
50%       0.928038
75%       0.955463
max       1.101962
Name: lowest, dtype: float64

In [28]:
.2/68

0.0029411764705882353

In [5]:
solver.result.argmin.head(10)

,prob,win_down,marg_cont,marg_info,win0,win_up,cost,metric
0,0.784545,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.188643,0.908844,0.004235,0.011783,0.820767,0.815524,0.750958,0.0
2,0.012786,0.500828,0.185925,0.439655,0.286486,0.285175,0.050533,0.0
3,0.006672,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.0
4,0.006672,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.0
5,0.000195,0.592042,0.009549,0.255264,0.576157,0.572252,0.208498,0.0
6,0.000078,0.133888,0.103488,0.388073,0.025210,0.023008,0.529429,0.0
7,0.000026,0.221396,0.004258,0.460932,0.085311,0.080532,0.976700,0.0
8,0.000023,0.355889,0.039753,0.516866,0.303292,0.295892,0.901691,0.0
9,0.000021,0.805127,0.012244,0.130930,0.779111,0.769247,0.459294,0.0


In [6]:
pb.variable.value[-5:]

array([[2.06364344e-14],
       [2.35776236e-14],
       [6.67223983e-03],
       [6.67223983e-03],
       [7.84545364e-01]])

In [7]:
c = pb.constraints[-1]
c.value()

True

In [8]:
pb._beliefs

array([[0.96366276, 0.05597224, 0.00499326, 0.891773  , 0.43758721],
       [0.92559664, 0.29191006, 0.00395948, 0.56804456, 0.07103606],
       [0.97861834, 0.08996824, 0.00244074, 0.87001215, 0.77815675],
       ...,
       [1.        , 1.        , 1.        , 0.        , 0.        ],
       [1.        , 1.        , 1.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])

In [9]:
demands = pb.variable.value.T @ pb._beliefs
demands

array([[0.19156584, 0.01656595, 0.02136348, 0.15879307, 0.15775808]])

In [10]:
multistage_moment_matrix @ demands.T

array([[-0.19156584],
       [-0.00552508],
       [ 0.15879307],
       [ 0.00103498],
       [-0.15775808]])

In [11]:
multistage_moment_matrix @ demands.T - solver.tolerance

/home/sylvain/.virtualenvs/py38-workspace/lib/python3.8/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/sylvain/.virtualenvs/py38-workspace/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


array([[-7.54549127e-05],
       [-4.41663260e-05],
       [-7.31529960e-02],
       [-1.21734650e-07],
       [-6.03180591e-05]])